In [1]:
import torch.optim as optim
from ray import tune
from ray.tune import track
from ray.tune.schedulers import ASHAScheduler
from ray.tune.examples.mnist_pytorch import get_data_loaders, ConvNet, train, test

from dpdqn_v2 import DPDQN2
import os
import gym
import numpy as np
import matplotlib.pyplot as plt

from shutil import copyfile
from utils import *
import pickle
import pandas as pd

import datetime

In [2]:
# test 1 on server

In [3]:
%%time
exp_name="server_tune_" + datetime.datetime.now().strftime("%Y_%B_%d_%I_%M%p")

def train_DPDQN(config):
    time_steps = 1e6 #testrun
    envname = "BipedalWalker-v2"
    env = gym.make(envname)
    model = DPDQN2(env, verbose=0, ray_tune=True, 
                  batch_size=config["batch_size"], num_action_samples_final=config["num_action_samples_final"],
                   hidden_size2=config["hidden_size2"], opt_steps_per_step=config["opt_steps_per_step"],
                   gamma=config["gamma"], replay_size=config["replay_size"]
                  )
    model.learn(total_timesteps=int(time_steps))
    

search_space={
    "batch_size": tune.grid_search([16, 32]),
    "num_action_samples_final": tune.grid_search([11, 15, 19]),
    "hidden_size2": tune.grid_search([16, 32]),
    "opt_steps_per_step": tune.grid_search([1, 2]),
    "gamma": tune.grid_search([0.99, 0.995]),
    "replay_size": tune.grid_search([5000, 7500]),
}

analysis = tune.run(
    train_DPDQN,
    name=exp_name,
    config=search_space,
    resources_per_trial={"cpu": 1, "gpu": 0}
)

print("Best config: ", analysis.get_best_config(metric="mean_reward"))

dfs = analysis.trial_dataframes
pickle.dump(dfs, open("tune_results.p", "wb"), pickle.HIGHEST_PROTOCOL)


#tensorboard --logdir ~/ray_results

2019-11-18 04:02:24,033	INFO resource_spec.py:205 -- Starting Ray with 3.66 GiB memory available for workers and up to 1.84 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2019-11-18 04:02:24,826	INFO function_runner.py:254 -- tune.track signature detected.


== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/12 CPUs, 0/0 GPUs, 0.0/3.66 GiB heap, 0.0/1.22 GiB objects
Memory usage on this node: 9.9/16.0 GiB

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 1/12 CPUs, 0/0 GPUs, 0.0/3.66 GiB heap, 0.0/1.22 GiB objects
Memory usage on this node: 9.8/16.0 GiB
Result logdir: /Users/aydarakhmetzyanov/ray_results/server_tune_2019_November_18_04_02AM
Number of trials: 12 ({'RUNNING': 1, 'PENDING': 11})
PENDING trials:
 - train_DPDQN_1_batch_size=32,hidden_size2=16,num_action_samples_final=11:	PENDING
 - train_DPDQN_2_batch_size=16,hidden_size2=32,num_action_samples_final=11:	PENDING
 - train_DPDQN_3_batch_size=32,hidden_size2=32,num_action_samples_final=11:	PENDING
 - train_DPDQN_4_batch_size=16,hidden_size2=16,num_action_samples_final=13:	PENDING
 - train_DPDQN_5_batch_size=32,hidden_size2=16,num_action_samples_final=13:	PENDING
 - train_DPDQN_6_batch_size=16,hidden_size2=32,num_action_samples_final=13:	PENDING
 

Result for train_DPDQN_9_batch_size=32,hidden_size2=16,num_action_samples_final=16:
  date: 2019-11-18_04-03-06
  done: false
  experiment_id: 3082360a143b437db21c1c8bfb0b6702
  hostname: Aydars-MacBook-Pro.local
  iterations_since_restore: 1
  mean_reward: -114.06306487535944
  node_ip: 192.168.1.66
  pid: 55416
  time_since_restore: 35.26478314399719
  time_this_iter_s: 35.26478314399719
  time_total_s: 35.26478314399719
  timestamp: 1574038986
  timesteps_since_restore: 0
  training_iteration: 0
  trial_id: 153bfd54
  
== Status ==
Using FIFO scheduling algorithm.
Resources requested: 12/12 CPUs, 0/0 GPUs, 0.0/3.66 GiB heap, 0.0/1.22 GiB objects
Memory usage on this node: 9.0/16.0 GiB
Result logdir: /Users/aydarakhmetzyanov/ray_results/server_tune_2019_November_18_04_02AM
Number of trials: 12 ({'RUNNING': 12})
RUNNING trials:
 - train_DPDQN_0_batch_size=16,hidden_size2=16,num_action_samples_final=11:	RUNNING
 - train_DPDQN_1_batch_size=32,hidden_size2=16,num_action_samples_final=11:

Result for train_DPDQN_2_batch_size=16,hidden_size2=32,num_action_samples_final=11:
  date: 2019-11-18_04-14-47
  done: false
  experiment_id: c0b4083bbe124f8cbce6fb5e075fe5fc
  hostname: Aydars-MacBook-Pro.local
  iterations_since_restore: 2
  mean_reward: -105.35629097331095
  node_ip: 192.168.1.66
  pid: 55410
  time_since_restore: 736.8602771759033
  time_this_iter_s: 729.0946052074432
  time_total_s: 736.8602771759033
  timestamp: 1574039687
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: '15399546'
  
Result for train_DPDQN_4_batch_size=16,hidden_size2=16,num_action_samples_final=13:
  date: 2019-11-18_04-14-48
  done: false
  experiment_id: 0ef86e709afe4263ac0abdac6b93924c
  hostname: Aydars-MacBook-Pro.local
  iterations_since_restore: 2
  mean_reward: -109.19338276311589
  node_ip: 192.168.1.66
  pid: 55409
  time_since_restore: 738.0922122001648
  time_this_iter_s: 730.7106289863586
  time_total_s: 738.0922122001648
  timestamp: 1574039688
  timesteps_since_r

Result for train_DPDQN_5_batch_size=32,hidden_size2=16,num_action_samples_final=13:
  date: 2019-11-18_04-25-49
  done: false
  experiment_id: ef8b1ad2e0d6438fa527cbceadf74c78
  hostname: Aydars-MacBook-Pro.local
  iterations_since_restore: 2
  mean_reward: -115.52700013384826
  node_ip: 192.168.1.66
  pid: 55406
  time_since_restore: 1398.6813778877258
  time_this_iter_s: 1391.590142250061
  time_total_s: 1398.6813778877258
  timestamp: 1574040349
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 153ac0c6
  
== Status ==
Using FIFO scheduling algorithm.
Resources requested: 12/12 CPUs, 0/0 GPUs, 0.0/3.66 GiB heap, 0.0/1.22 GiB objects
Memory usage on this node: 10.2/16.0 GiB
Result logdir: /Users/aydarakhmetzyanov/ray_results/server_tune_2019_November_18_04_02AM
Number of trials: 12 ({'RUNNING': 12})
RUNNING trials:
 - train_DPDQN_0_batch_size=16,hidden_size2=16,num_action_samples_final=11:	RUNNING, [1 CPUs, 0 GPUs], [pid=55412], 794 s, 1 iter
 - train_DPDQN_1_batch_siz

...

Result for train_DPDQN_8_batch_size=16,hidden_size2=16,num_action_samples_final=16:
  date: 2019-11-18_09-19-45
  done: false
  experiment_id: cc5550787c3a498998f9800b792c2ad6
  hostname: Aydars-MacBook-Pro.local
  iterations_since_restore: 27
  mean_reward: -104.63171351597585
  node_ip: 192.168.1.66
  pid: 55411
  time_since_restore: 19034.85214495659
  time_this_iter_s: 701.5038101673126
  time_total_s: 19034.85214495659
  timestamp: 1574057985
  timesteps_since_restore: 0
  training_iteration: 26
  trial_id: 153bacf0
  
== Status ==
Using FIFO scheduling algorithm.
Resources requested: 12/12 CPUs, 0/0 GPUs, 0.0/3.66 GiB heap, 0.0/1.22 GiB objects
Memory usage on this node: 10.2/16.0 GiB
Result logdir: /Users/aydarakhmetzyanov/ray_results/server_tune_2019_November_18_04_02AM
Number of trials: 12 ({'RUNNING': 12})
RUNNING trials:
 - train_DPDQN_0_batch_size=16,hidden_size2=16,num_action_samples_final=11:	RUNNING, [1 CPUs, 0 GPUs], [pid=55412], 18582 s, 26 iter
 - train_DPDQN_1_batch_

Result for train_DPDQN_10_batch_size=16,hidden_size2=32,num_action_samples_final=16:
  date: 2019-11-18_09-28-15
  done: false
  experiment_id: cff57bd2b0bc400a9cb55615301ec360
  hostname: Aydars-MacBook-Pro.local
  iterations_since_restore: 28
  mean_reward: -109.21514476476021
  node_ip: 192.168.1.66
  pid: 55405
  time_since_restore: 19544.51454114914
  time_this_iter_s: 712.3281629085541
  time_total_s: 19544.51454114914
  timestamp: 1574058495
  timesteps_since_restore: 0
  training_iteration: 27
  trial_id: 153c4868
  
== Status ==
Using FIFO scheduling algorithm.
Resources requested: 12/12 CPUs, 0/0 GPUs, 0.0/3.66 GiB heap, 0.0/1.22 GiB objects
Memory usage on this node: 10.4/16.0 GiB
Result logdir: /Users/aydarakhmetzyanov/ray_results/server_tune_2019_November_18_04_02AM
Number of trials: 12 ({'RUNNING': 12})
RUNNING trials:
 - train_DPDQN_0_batch_size=16,hidden_size2=16,num_action_samples_final=11:	RUNNING, [1 CPUs, 0 GPUs], [pid=55412], 19287 s, 27 iter
 - train_DPDQN_1_batch

Result for train_DPDQN_3_batch_size=32,hidden_size2=32,num_action_samples_final=11:
  date: 2019-11-18_09-33-21
  done: false
  experiment_id: 722ab315039f4cb8bace0efa764e50d8
  hostname: Aydars-MacBook-Pro.local
  iterations_since_restore: 16
  mean_reward: -99.25133082946736
  node_ip: 192.168.1.66
  pid: 55414
  time_since_restore: 19851.15104985237
  time_this_iter_s: 1303.2595148086548
  time_total_s: 19851.15104985237
  timestamp: 1574058801
  timesteps_since_restore: 0
  training_iteration: 15
  trial_id: 153a060c
  
== Status ==
Using FIFO scheduling algorithm.
Resources requested: 12/12 CPUs, 0/0 GPUs, 0.0/3.66 GiB heap, 0.0/1.22 GiB objects
Memory usage on this node: 10.3/16.0 GiB
Result logdir: /Users/aydarakhmetzyanov/ray_results/server_tune_2019_November_18_04_02AM
Number of trials: 12 ({'RUNNING': 12})
RUNNING trials:
 - train_DPDQN_0_batch_size=16,hidden_size2=16,num_action_samples_final=11:	RUNNING, [1 CPUs, 0 GPUs], [pid=55412], 19287 s, 27 iter
 - train_DPDQN_1_batch_

Result for train_DPDQN_0_batch_size=16,hidden_size2=16,num_action_samples_final=11:
  date: 2019-11-18_09-35-57
  done: false
  experiment_id: 9c15448bff3945339c445aef3f1fd23d
  hostname: Aydars-MacBook-Pro.local
  iterations_since_restore: 29
  mean_reward: -55.73752764215061
  node_ip: 192.168.1.66
  pid: 55412
  time_since_restore: 20006.430218935013
  time_this_iter_s: 718.4626898765564
  time_total_s: 20006.430218935013
  timestamp: 1574058957
  timesteps_since_restore: 0
  training_iteration: 28
  trial_id: '15354368'
  
== Status ==
Using FIFO scheduling algorithm.
Resources requested: 12/12 CPUs, 0/0 GPUs, 0.0/3.66 GiB heap, 0.0/1.22 GiB objects
Memory usage on this node: 10.4/16.0 GiB
Result logdir: /Users/aydarakhmetzyanov/ray_results/server_tune_2019_November_18_04_02AM
Number of trials: 12 ({'RUNNING': 12})
RUNNING trials:
 - train_DPDQN_0_batch_size=16,hidden_size2=16,num_action_samples_final=11:	RUNNING, [1 CPUs, 0 GPUs], [pid=55412], 20006 s, 28 iter
 - train_DPDQN_1_bat

RayletError: The Raylet died with this message: [RayletClient] Raylet connection closed.

In [4]:
#dfs=pickle.load(open("tune_results.p", "rb"))
ax = None  # This plots everything on the same plot
for d in dfs.values():
    ax = d.mean_accuracy.plot(ax=ax, legend=False)
    
#tensorboard --logdir ~/ray_results

NameError: name 'dfs' is not defined

In [ ]:
#tensorboard  --host 0.0.0.0 —port 6006 --logdir ~/ray_results/server_tune_2019_November_14_10_10PM